In [4]:
import requests
import datetime
import pandas as pd
import xml.etree.ElementTree as ET

pd.options.plotting.backend = "plotly"

def ParseDateTime(date):
    divide = date.split(".")
    return datetime.datetime(int(divide[2]), int(divide[1]), int(divide[0]))

def GetValutaCodes():
    url = "https://cbr.ru/scripts/XML_val.asp?d=0"

    response = requests.post(url)
    root = ET.fromstring(response.content)

    result = {}

    for child in root.findall("Item"):
        result[child.find("ParentCode").text.replace(" ", "")] = child.find("Name").text

    return result

dateFrom = "01/08/2004"
dateTo = "02/09/2024"
valutaCode = "R01239"
valuta = GetValutaCodes()[valutaCode]

url = f"https://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={dateFrom}&date_req2={dateTo}&VAL_NM_RQ={valutaCode}"

response = requests.post(url)

root = ET.fromstring(response.content)

t = {}

for child in root.findall("Record"):
    t[ParseDateTime(child.attrib["Date"])] = float(child.find('Value').text.replace(',', '.')) / int(child.find("Nominal").text)

ts = pd.DataFrame.from_dict(t, orient= "index", columns= [valuta])

fig = ts.plot(title=f"Российский рубль - {valuta}", template= "plotly_dark", labels=dict(index="Дата", value="Курс", variable="Валюта"))
fig.update_yaxes(tickprefix="₽")
fig.show()